In [2]:
from transformers import pipeline, set_seed
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt

from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import evaluate 
import json
import ijson
import nltk
from nltk.tokenize import sent_tokenize
import os
import pandas as pd
import random
from rouge_score import rouge_scorer
from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zacha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!pip install ijson


     ---------------------------------------- 48.2/48.2 kB 2.5 MB/s eta 0:00:00


In [14]:
#open loco json file
# Get the current script directory
filename='loco.json'
# file_dir = os.path.dirname(__file__)
# print(file_dir)
# # Construct the relative path to the JSON file
# file_path = os.path.join( 'data',filename  )
file_path = 'data/LOCO.json'
# file_path
# with open(file_path, 'rb') as f:
#     record = next(ijson.items(f, "item")) # get the first record
#     print(record) 
df=pd.read_json(file_path)

df = pd.DataFrame(df)

df.head


    

<bound method NDFrame.head of        doc_id                                                URL  \
0      C00001  https://humansarefree.com/2016/12/the-conspira...   
1      C00003  https://americandigitalnews.com/index.php/2017...   
2      C00004  https://911truth.org/whats-crazier-believing-t...   
3      C00005  https://rense.com/general54/cancer-causing_vac...   
4      C00007  https://awarenessact.com/scientist-shares-impo...   
...       ...                                                ...   
96738  M20fd0  https://uk.reuters.com/article/us-health-coron...   
96739  M20fd1  https://www.pbs.org/frontlineworld/stories/ira...   
96740  M20fd4  https://science.howstuffworks.com/life/remembe...   
96741  M20fd8  https://theconversation.com/mind-control-skill...   
96742  M20fda  https://www.nme.com/news/music/george-michael-...   

                       website                        seeds       date  \
0            humansarefree.com        michael.jackson.death 2016-12-30   
1    

In [15]:
lfs_file_path='data/LOCO_LFs.json'

# file_path
# with open(file_path, 'rb') as f:
#     record = next(ijson.items(f, "item")) # get the first record
#     print(record) 
df_lfs=pd.read_json(lfs_file_path)

df_lfs = pd.DataFrame(df_lfs)

df_lfs.head

<bound method NDFrame.head of        doc_id  LIWC_WC  LIWC_Analytic  LIWC_Clout  LIWC_Authentic  LIWC_Tone  \
0      C00001     4051          88.89       76.97           13.98       5.22   
1      C00003      929          99.00       68.20           10.45      37.18   
2      C00004      706          90.03       53.40           29.13      25.77   
3      C00005     4989          95.62       63.26           13.75      22.05   
4      C00007      387          30.31       93.94            9.63      73.97   
...       ...      ...            ...         ...             ...        ...   
96738  M20fd0      767          89.83       75.94           12.00      27.94   
96739  M20fd1      333          97.38       61.82           44.17       1.09   
96740  M20fd4      206          87.48       78.14           52.39      12.81   
96741  M20fd8      514          94.28       71.37           15.01      47.35   
96742  M20fda      320          78.95       66.91           18.52      10.13   

       LI

In [16]:
topic_gamma_file_path='data/topic_gamma.json'

# file_path
# with open(file_path, 'rb') as f:
#     record = next(ijson.items(f, "item")) # get the first record
#     print(record) 
df_topic_gamma=pd.read_json(lfs_file_path)

df_topic_gamma = pd.DataFrame(df_topic_gamma)

df_topic_gamma.head

<bound method NDFrame.head of        doc_id  LIWC_WC  LIWC_Analytic  LIWC_Clout  LIWC_Authentic  LIWC_Tone  \
0      C00001     4051          88.89       76.97           13.98       5.22   
1      C00003      929          99.00       68.20           10.45      37.18   
2      C00004      706          90.03       53.40           29.13      25.77   
3      C00005     4989          95.62       63.26           13.75      22.05   
4      C00007      387          30.31       93.94            9.63      73.97   
...       ...      ...            ...         ...             ...        ...   
96738  M20fd0      767          89.83       75.94           12.00      27.94   
96739  M20fd1      333          97.38       61.82           44.17       1.09   
96740  M20fd4      206          87.48       78.14           52.39      12.81   
96741  M20fd8      514          94.28       71.37           15.01      47.35   
96742  M20fda      320          78.95       66.91           18.52      10.13   

       LI